In [7]:
import numpy as np
import pandas
import tensorflow as tf
from importlib import reload
import graph; reload(graph)

<module 'graph' from '/home/stassyvichev/w266-final-project/graph.py'>

In [2]:
# LOAD THE DATA
fileName = "data/encoded_data/embeddings_w266.npy"
encodedData = np.load(fileName)
encodedData = pandas.DataFrame(data = encodedData, columns = ["x_"+str(i) for i in range(500)])
filename = "data/Labeled_Colorado_Flu_Study_Tweets_AvI_RvN_SvO.csv"
coloradoData = pandas.read_csv(filename, sep="\t")
print(encodedData.shape)
allData = pandas.concat([coloradoData, encodedData], axis = 1)
print(allData.shape)

(5270, 500)
(5270, 505)


In [32]:
allData

,Tweet_ID,Tweet_Content,Awareness_Label,Related_Label,Self_Label,x_0,x_1,x_2,x_3,x_4,...,x_490,x_491,x_492,x_493,x_494,x_495,x_496,x_497,x_498,x_499
0,5926297601,"don't worry it's not swine flu, i already got ...",1.0,NaN,1.0,0.042112,-0.250500,-0.225909,0.302922,-0.115346,...,0.822675,0.459677,-1.262836,0.363780,1.816308,-0.658702,0.388301,-0.109658,-1.368834,-1.720524
1,5222838706,muh. if i am getting sick and it's not swine f...,0.0,1.0,1.0,-0.174253,0.130687,-1.384781,0.459587,-0.777744,...,0.548753,1.287073,0.312468,-0.465325,2.188294,-0.027273,0.113580,-1.535737,-0.806254,-1.206642
2,5233654812,what is up with my boy? this morning i though...,0.0,NaN,0.0,1.596801,0.250821,0.436145,-2.845884,-0.464014,...,0.060714,0.973608,0.157539,-0.807876,0.682885,-2.189883,1.119996,1.225436,0.711882,-1.237789
3,5918860304,"getting better,no more piggy flu 4 me,it was n...",0.0,1.0,1.0,2.034387,0.432776,-0.038979,-0.650743,0.067163,...,0.699051,1.544988,0.092756,-1.447382,0.576435,1.077597,-0.316096,1.288783,-1.124577,-0.827067
4,4631607800,@robbsterr yay for man txting you.. in other n...,0.0,0.0,1.0,-0.469187,1.415714,-1.875996,0.901807,0.076518,...,0.744410,0.573868,0.070829,-0.891997,0.930053,-0.876152,0.057401,-0.118646,0.211558,-1.097714
5,6004314210,swine flu has arrived at my daughter's kinderg...,0.0,1.0,0.0,-0.658460,1.165405,-0.449825,-0.973872,-0.883874,...,1.596636,0.630788,-2.392556,-2.039080,0.553627,-1.375080,0.043998,-0.833872,-0.364098,-1.476711
6,5225053106,i think im getting flu soon.,0.0,1.0,1.0,0.300915,-0.538062,-0.130646,1.250716,0.042007,...,0.164920,0.193372,-0.313867,0.543114,1.619272,-0.723616,-0.059287,-0.809263,1.339956,-1.840384
7,5229704707,h1n1 flu is rampant in nagasaki. holy crap. 70...,0.0,NaN,0.0,0.916144,0.704700,-0.177146,0.486754,-1.278458,...,0.325460,-0.037231,-0.086262,0.056054,-0.823911,-0.130337,1.364163,-1.321458,0.098823,-2.450318
8,5233811803,rt @adamwsyx6: getting my seasonal flu shot to...,1.0,NaN,1.0,-0.323790,1.581128,0.851611,-0.449899,-1.564968,...,1.779564,-0.270962,-1.629676,-1.735691,1.109505,0.379852,-1.213338,-0.256298,-0.487033,-1.800045
9,5900297914,@sihawkings you like christmas then? i'll send...,0.0,NaN,0.0,0.886542,1.922134,0.480182,-1.645274,-0.516601,...,-0.378339,0.230550,-1.808538,0.124601,0.429019,0.452213,-1.022731,-1.170298,0.067770,-1.558405


In [42]:
# get all data with labels present for the column we care about
allDataVal = allData.dropna(subset=["Related_Label"])
print(allDataVal.shape)

# extract X and Y as np arrays, so that we can feed them to tensors. 
x = allDataVal[["x_"+str(i) for i in range(500)]]
y = allDataVal["Related_Label"]
print(x.values.shape)
print(y.values.shape)

(4413, 505)
(4413, 500)
(4413,)


In [60]:
# split into train, test, dev (80%,10%,10%)
np.random.seed(42)
train_x, test_x, dev_x = np.split(x.sample(frac=1), [int(.8*len(x)), int(.9*len(x))])
train_y = y[train_x.index]
test_y = y[test_x.index]
dev_y = y[dev_x.index]

# convert to numpy arrays
train_x, test_x, dev_x, train_y, test_y, dev_y = train_x.values, test_x.values, dev_x.values, train_y.values, test_y.values, dev_y.values 

# our vector size is static, as it is already trained
max_len = 500
train_ns = np.full((train_x.shape[0],1),max_len)
test_ns = np.full((test_x.shape[0],1),max_len)
test_ns = np.full((test_x.shape[0],1),max_len)
print(type(train_x))

<class 'numpy.ndarray'>


In [61]:
# create tensors for x_ and y_, not sure if needed; TODO do we need to create the other ones as well?
with tf.name_scope("Inputs"):
    x_ = tf.convert_to_tensor(train_x, dtype = tf.float32, name = "x_")
    y_ = tf.convert_to_tensor(train_y, dtype = tf.float32, name = "y_")
    print(x_)
    print(y_)

Tensor("Inputs_1/x_:0", shape=(3530, 500), dtype=float32)
Tensor("Inputs_1/y_:0", shape=(3530,), dtype=float32)
